# Transfer Learning을 통한 꽃 사진 분류

## 설치하기
### Windows

```bash
conda create —-name=book-trasnfer-learning python=3.5
conda activate book-transfer-learning
conda install jupyter notebook tqdm tensorflow scikit-learn matplotlib scikit-image
git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
```

### OS X

```bash
conda create -—name=book-trasnfer-learning python=3.5
source activate book-transfer-learning
conda install jupyter notebook tqdm tensorflow scikit-learn matplotlib scikit-image
git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
```

## 문제 소개

이번 장에서는 꽃사진 분류를 위한 딥러닝 모델을 만드는 것이다. 텐서플로가 제공해주는 꽃사진 데이터셋을 이용한다. 총 5가지의 꽃종류가 폴더 별로 정리되어 있다. `flower_photos`안에는 다음과 같은 폴더들이 있다.

- `daisy` : 데이지
- `dandelion` : 민들레
- `roses` : 장미
- `sunflowers` : 해바라기
- `tulips` : 튤립

일반적으로 꽃을 분류하는 모델을 만드려면 여러 사항을 고려해야한다. 먼저, 신경망 모델을 사용할지, CNN 모델을 사용할지 혹은 그외 모델을 사용할지 정해야한다. 게다가 히든 레이어는 몇개를 만들 것인지, activation 함수는 무엇을 사용할 것인지 등 많은 세부사항들을 결정해야 한다.

이렇게 많은 고민 끝에 모델을 결정했다고 하더라도, 학습시키는데 어려운 점들이 있지만, 가장 어려운점은 학습시간이 매우 오래 걸린다는 점이다. 정확한 결과를 위해서는 복잡한 모델을 선택하게 되므로, 학습시간이 길어지는 현상은 피할 수 없다. 논문이나 언론에서 접하는 모델들을 학습시키기 위해 1주를 넘어가는 일은 흔한 일이다. 

이러한 어려운 점들을 Transfer Learning을 통해 극복 할 수 있다. Transfer Learning이란, 기존에 잘 만들어 놓은 딥러닝 모델을 우리가 만드록 싶은 모델에 전이(Transfer)시키는 학습 기술이다. 즉, 기존 모델을 이용하여, 우리는 최소한의 리소스와 시간으로, 최대한의 정확도를 만드는 방법이다.

이번장에서는 Transfer Learning을 사용하여, ImageNet 데이터셋으로 학습한 VGG16 모델을 사용하여, 꽃사진 분류 모델을 학습시킬 것이다. 

## 모델 이해에 필요한 개념

### ImageNet

ImageNet[^1]은 물체 인식 소프트웨어 개발을 위해 만들어진 프로젝트이다. 데이터셋은 약 2만개 종류의 물체들이 약1천4백만장의 이미지의 URL들로 구성되어 있다. Kaggle에서 이 데이터셋을 이용하는 Challenge가 존재해서, 많은 사람들이 뛰어들어 연구되고 있는 데이터셋이다.

[^1]: 더 자세한 사항이 궁금하다면 다음 사이트를 참고하면 된다.(http://www.image-net.org)

### VGG16

이번장에서는 ImageNet을 사용하는 많은 모델중에 VGG16을 이용할 것이다.

![VGG16 모델](images/original_vgg16.png)

그림에서 볼 수 있듯이, 기본적으로 CNN 모델이다. Convolution 레이어가 13개나 있는 매우 깊은 모델이다. 이런 복잡하고 깊은 모델을 학습 시키려면 아주 많은 데이터셋이 필요하고, 학습시키는데 많은 시간이 소요된다. 꽃분류 문제의 해결을 위해서 VGG16 모델들 중에 다음 Github 저장소에 공개된 모델을 사용한다.

> https://github.com/machrisaa/tensorflow-vgg

### Transfer Learning과 Bottleneck 특성

Transfer Learning은 간단하다. 기존의 VGG16모델에서 유지하고 싶은 부분을 정한뒤, 그 뒷부분의 레이어들은 삭제하고 특성(Feature)들을 뽑으면 된다.
필자는 다음 그림처럼 뒤의 2가지 Dense 레이어를 삭제시켰다. 

![VGG16 모델](images/cropped_vgg16.png)

## 모델 구조

Bottleneck 특성을 얻어낸 뒤, 기존의 Dense(4096) 2겹을 Dense(256)과 Dense(5)로 변경하여, 마지막 2개의 레이어만 학습시킨다.

![학습시킬 모델 구조](images/our_vgg16.png)

모델 구조만 보면 크게 달라진 점이 없지만, 학습의 관점에서 보면 매우 간소화 되었다. 기존의 VGG16 모델의 Weight값들을 그대로 사용하고, 마지막 2개의 레이어만 학습시키므로, 최적화 문제의 차원이 비교할 수 없을만큼 작아진 것이다.

## 데이터 훑어보기

다음 코드는 `flower_photos` 데이터셋을 다운로드하는 코드이다. 5가지 종류의 꽃의 사진 파일이 3,670개 다운로드 된다.

In [4]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

import tarfile
dataset_folder_path = 'flower_photos'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('flower_photos.tar.gz'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Flowers Dataset') as pbar:
        urlretrieve(
            'http://download.tensorflow.org/example_images/flower_photos.tgz',
            'flower_photos.tar.gz',
            pbar.hook)

if not isdir(dataset_folder_path):
    with tarfile.open('flower_photos.tar.gz') as tar:
        tar.extractall()
        tar.close()

Flowers Dataset: 229MB [01:08, 3.34MB/s]                              


다음은 필자가 미리 추출한 Bottleneck 특성을 불러오는 코드이다. 미리 추출한 파일로 진행하는 이유는 처리해야할 데이터가 많고, 모델이 복잡하여 계산에 많은 시간이 소요되어 독자들의 학습에 방해가 될 것 같다는 판단때문이다. 하지만 Bottleneck 특성을 추출하는 방법은 매우 중요하므로 생략하기보다는 사용한 코드를 이번 장의 마지막 절에 설명을 추가하였다.

`labels.txt`에는 꽃의 종류가 문자열로 표시되어 있다. `codes.bin`에는 가로, 세로 모두 243 픽셀인 이미지가 입력으로 들어가서, 앞서 VGG16 모델의 앞부분을 통과하여 Dense(4096)의 출력 벡터가 저장되어있다.

In [5]:
import numpy as np
import csv
with open('labels.txt') as f:
    reader = csv.reader(f, delimiter='\n')
    labels = np.array([each for each in reader if len(each) > 0]).squeeze()

with open('codes.bin') as f:
    codes = np.fromfile(f, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

다음과 같이 이미지 1장당 4,096차원 벡터 출력이고, 총 3,670개의 데이터셋을 로드하였다.

In [6]:
print(codes.shape)

(3670, 4096)


다음과 같이 `sklearn`의 `LabelBinarizer`를 사용하면, 범주형 데이터를 편리하게 One-hot 인코딩된 벡터로 변경할 수 있다.

In [7]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(labels)

labels_vecs = lb.transform(labels)

print(labels[[0, 1000, 2000, 2500, 3000]])
print(labels_vecs[[0, 1000, 2000, 2500, 3000]])

['daisy' 'dandelion' 'roses' 'sunflowers' 'tulips']
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


`sklearn`의 `StratifiedShuffleSplit`을 사용하면, 학습용 데이터와 테스트용 데이터를 편리하게 나눌 수 있다. `val_idx`는 테스트용 데이터의 인덱스를 나타내는 벡터다. 테스트용 데이터를 2개로 나눠서, 최종 테스트용과 Validation용 데이터셋을 만든다. 

`val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]`는 파이썬의 편리한 인덱스 사용법이다. `val_idx` 벡터 중 첫번째부터 `half_val_len`까지를 Validation용으로 분류하고(`val_idx[:half_val_len]`), `half_val_len`이후부터 마지막까지를 테스트용으로 분류하는 작업을 한줄로 표현한 것이다(`val_idx[half_val_len:]`).

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_idx, val_idx = next(ss.split(codes, labels_vecs))

half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (2936, 4096) (2936, 5)
Validation shapes (x, y): (367, 4096) (367, 5)
Test shapes (x, y): (367, 4096) (367, 5)


## 텐서플로

### 모델 만들기

입력과 출력을 `tf.placeholder`로 선언을 한다. Dense(256)과 Dense(5)는 `tf.layers.dense`로 편리하게 선언할 수 있다. 마지막 레이어에서는 Softmax를 사용하지 않고, Logit만 구한다(`activation=None`). 이러면, `tf.nn.softmax_cross_entropy_with_logits`를 사용하여 편리하게 크로스-엔트로피를 계산할수 있다.

In [9]:
import tensorflow as tf

inputs_ = tf.placeholder(tf.float32, shape=[None, codes.shape[1]])
labels_ = tf.placeholder(tf.float32, shape=[None, labels_vecs.shape[1]])

fc = tf.layers.dense(inputs_, 256, activation=tf.nn.relu)
print(fc)
logits = tf.layers.dense(fc, labels_vecs.shape[1], activation=None)
print(logits)

Tensor("dense/Relu:0", shape=(?, 256), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 5), dtype=float32)


### 최적화 문제 설정

`tf.nn.softmax_cross_entropy_with_logits`를 사용하여 크로스-엔트로피를 구한다. 최적화를 위한 수치알고리즘은 ADAM을 선택한다. 

In [10]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer().minimize(cost)

정확도 계산을 하기 위한 코드는 다음과 같다.

In [11]:
predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### 하이퍼 파라미터 설정

Batch의 크기는 16, 그리고 최대 Epoch은 10으로 설정한다.

In [12]:
batch_size = 16
epochs = 10

케라스와 다르게 텐서플로는 Batch를 만드는 코드를 직접 작성해야하는 경우가 많다. 다음과 같이 `get_batches(x, y, batch_size=16)`로 학습용 데이터와 라벨을 넣고, Batch의 크기를 정해주면, Batch들을 생성하는 기능을 만든다.

In [13]:
def get_batches(x, y, batch_size=16):
    """ Return a generator that yields batches from arrays x and y. """
    n_batches = np.int32(np.floor(len(x) / batch_size))
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        yield X, Y

### 학습 시작

앞에서 설정한 최적화 문제와 하이퍼 파라미터들을 이용하여, 학습을 진행한다. 학습을 다 진행한 후 모델을 `checkpoints/flowers.ckpt`의 파일로 저장한다.
이를 위한 폴더 `checkpoints`를 만들어야 에러가 발생하지 않는다.

In [14]:
!mkdir checkpoints

매 Epoch마다 Validation 데이터셋에서의 모델 성능을 표시해주는 코드가 `print`로 추가 되어 있다.

In [15]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y, batch_size):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
        feed = {inputs_: val_x,
                labels_: val_y}
        val_acc = sess.run(accuracy, feed_dict=feed)
        print("Epoch: {}/{}".format(e+1, epochs),
              "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/flowers.ckpt")

Epoch: 1/10 Validation Acc: 0.8420
Epoch: 2/10 Validation Acc: 0.8556
Epoch: 3/10 Validation Acc: 0.8692
Epoch: 4/10 Validation Acc: 0.8719
Epoch: 5/10 Validation Acc: 0.8719
Epoch: 6/10 Validation Acc: 0.8883
Epoch: 7/10 Validation Acc: 0.8692
Epoch: 8/10 Validation Acc: 0.8529
Epoch: 9/10 Validation Acc: 0.8583
Epoch: 10/10 Validation Acc: 0.8937


### 정확도 확인

최종 정확도는 초기값의 무작위 설정으로 인해 약간 다를 수 있지만, 대략 90%의 성능을 얻을 수 있다.

In [16]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints/flowers.ckpt
Test accuracy: 0.8965


## Bottleneck 특성 추출 코드

### 텐서플로 

In [ ]:
vgg_dir = 'tensorflow_vgg/'
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(vgg_dir + "vgg16.npy"):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
            vgg_dir + 'vgg16.npy',
            pbar.hook)
else:
    print("Parameter file already exists!")

from tensorflow_vgg import vgg16
# Set the batch size higher if you can fit in in your GPU memory
batch_size = 16
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)

    for each in classes:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            # utils.load_image crops the input images for us, from the center
            img = utils.load_image(os.path.join(class_path, file))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # Running the batch through the network to get the codes
            if ii % batch_size == 0 or ii == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)# reshape
                
                # Here I'm building an array of the codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # Reset to start building the next batch
                batch = []
                print('{} images processed'.format(ii))

In [28]:
# write codes to file
with open('codes.bin', 'w') as f:
    codes.tofile(f)
    
# write labels to file
import csv
with open('labels.txt', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)